In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy import stats
from scipy import sparse
import scipy
from collections import defaultdict
import implicit
from implicit.als import AlternatingLeastSquares
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVDpp
import rectorch
from rectorch.models.baseline import SLIM, Random, Popularity
from rectorch.models.mf import EASE
from rectorch.data import DataProcessing
from rectorch.samplers import ArrayDummySampler, SparseDummySampler
from rectorch.evaluation import evaluate
from rectorch.utils import collect_results, prepare_for_prediction

# Data Parsing

In [3]:
# initialize data
item_threshold = 1 # used to filter out user/artist pairs that have been 
                   #listened to less than the threshold number of times
popular_artist_fraction = 0.2 # top cutoff for what we consider popular artists, in this case the top 20%

user_events_file = 'data/user_events.txt'
low_user_file = 'data/low_main_users.txt'
medium_user_file = 'data/medium_main_users.txt'
high_user_file = 'data/high_main_users.txt'

In [4]:
#read in user events file
cols = ['user', 'artist', 'album', 'track', 'timestamp']
df_events = pd.read_csv(user_events_file, sep='\t', names=cols)
print('No. of user events: ' + str(len(df_events)))
df_events.head() # check it is all read in properly

No. of user events: 28718087


,user,artist,album,track,timestamp
0,31435741,2,4,4,1385212958
1,31435741,2,4,4,1385212642
2,31435741,2,4,4,1385212325
3,31435741,2,4,4,1385209508
4,31435741,2,4,4,1385209191


## User-Artist Matrix

In [5]:
# create unique user-artist matrix
df_events = df_events.groupby(['user', 'artist']).size().reset_index(name='listens')
print('No. user-artist pairs: ' + str(len(df_events)))
# each row contains a unique user-artist pair, along with how many times the
# user has listened to the artist
#df_events.sort_values(by=['user'])

No. user-artist pairs: 1755361


In [6]:
df_events.head(1400)

,user,artist,listens
0,1021445,12,43
1,1021445,16,1
2,1021445,28,7
3,1021445,29,1
4,1021445,46,1
...,...,...,...
1395,1021445,1864220,3
1396,1021445,1864221,4
1397,1021445,1864222,1
1398,1045479,3,9


In [7]:
# filters out artist/user pairs who havent been listened two more than
# item_threshold amount of times to reduce potential load
# kept to 1 currently, so we dont filter out any data
df_events = df_events[df_events['listens'] >= item_threshold] 

# With 1, we see no difference between user-artist pairs here
print('No. filtered user-artist pairs: ' + str(len(df_events))) 

# here, we see the number of unique artists in our matrix
print('No. unique artists: ' + str(len(df_events['artist'].unique())))

No. filtered user-artist pairs: 1755361
No. unique artists: 352805


#### How many artists have users listened to?

In [8]:
# get matrix where each row is a user-id and how many artists they've 
#listened to
user_dist = df_events['user'].value_counts() 

# counts how many unique users there are. prints out user id & a count of how 
# many rows they're included in, which effectively shows how many artists 
# they listen to
num_users = len(user_dist)
print('Mean artists of all users: ' + str(user_dist.mean()))
print('Min artists of all users: ' + str(user_dist.min()))
print('Max artists of all users: ' + str(user_dist.max()))

user_dist.head()

Mean artists of all users: 585.1203333333333
Min artists of all users: 18
Max artists of all users: 4011


41888522    4011
4393555     3700
40029632    3678
26874346    3544
29736410    3529
Name: user, dtype: int64

#### How many users listen to an artist?

In [9]:
# get artist distribution
# same as previous but with artists, shows artist-id and how many times they
# were listened to buy unique users
artist_dist = df_events['artist'].value_counts()
num_artists = len(artist_dist)
print('No. artists: ' + str(num_artists))
df_events['artist'].value_counts().head

No. artists: 352805


<bound method NDFrame.head of 135        1389
1602       1359
46         1325
320        1297
27         1290
           ... 
3124286       1
1029181       1
1023032       1
1008679       1
3087545       1
Name: artist, Length: 352805, dtype: int64>

In [10]:
# get number of  popular artists
num_top_artists = int(popular_artist_fraction * num_artists)

# getting the top top_fraction (0.2) percent of artists, so finding how many
# artists make up 20% of total artists, and then only using the artists those
#number of the most popular aritsts
top_artist_dist = artist_dist[:num_top_artists]
print('No. top artists: ' + str(len(top_artist_dist)))

No. top artists: 70561


In [11]:
# read in users
# user file is just user_id and their mainstreaminess value 
low_users = pd.read_csv(low_user_file, sep=',').set_index('user_id')
medium_users = pd.read_csv(medium_user_file, sep=',').set_index('user_id')
high_users = pd.read_csv(high_user_file, sep=',').set_index('user_id')
num_users = len(low_users) + len(medium_users) + len(high_users)
print('Num users: ' + str(num_users))

Num users: 3000


## Getting Users From Each Popularity Group & Their 10 Most Listened To Artists 

### (For Analysis of Streaming Service Algorithmic Bias)

In [12]:
toList = df_events.loc[df_events['user'] == 42845367].sort_values(by=['listens'], ascending=False)
toList.head() #grabbing random users top 10 artists in 1 of the 3 groups

,user,artist,listens
1513514,42845367,495,205
1513639,42845367,15624,201
1513657,42845367,27107,171
1513483,42845367,163,156
1513485,42845367,172,156


In [13]:
to_list_2 = toList['artist'].tolist()[:20]
print(to_list_2)

[495, 15624, 27107, 163, 172, 4311, 8047, 7, 56, 245, 52741, 173416, 140, 1703, 3887, 42794, 1092, 2558, 54, 42835]


# Calculating GAP of User Profiles

In [14]:
# placeholder vars for numerator of GAPp, waiting to be divided by sie of group
low_gap_p = 0
medium_gap_p = 0
high_gap_p = 0
total_gap_p = 0
#Count for sanity check
low_count = 0
med_count = 0
high_count = 0

for u, df in df_events.groupby('user'):
    
    no_user_artists = len(set(df['artist'])) # profile size //number of artists in users profile
    # get popularity (= fraction of users interacted with item) of user items and calculate average of it
    user_pop_artist_fraq = sum(artist_dist[df['artist']] / num_users) / no_user_artists 
    
    if u in low_users.index: # get user group-specific values
        low_gap_p += user_pop_artist_fraq
        low_count += 1
    elif u in medium_users.index:
        medium_gap_p += user_pop_artist_fraq
        med_count += 1
    else:
        high_gap_p += user_pop_artist_fraq
        high_count += 1

total_gap_p = (low_gap_p + medium_gap_p + high_gap_p) / num_users
low_gap_p /= len(low_users) # average popularity of items/artists in low/med/high groups (gap = group average popularity)
medium_gap_p /= len(medium_users)
high_gap_p /= len(high_users)
print('Low count (for check): ' + str(low_count))
print('Med count (for check): ' + str(med_count))
print('High count (for check): ' + str(high_count))
print(low_gap_p)
print(medium_gap_p)
print(high_gap_p)

Low count (for check): 1000
Med count (for check): 1000
High count (for check): 1000
0.04963328792099549
0.054371119359489226
0.06286028679778642


### Min-Max Scaling Ratings (Not Using Right Now)

In [14]:
### Scale listening counts on a scale from 1-1000
"""scaled_df_events = pd.DataFrame()
for user_id, group in df_events.groupby('user'):
    #print(group)
    min_listens = group['listens'].min()
    max_listens = group['listens'].max()
    std = (group['listens'] - min_listens) / (max_listens - min_listens)
    scaled_listens = std * 999 + 1
    to_replace = group.copy()
    to_replace['listens'] = scaled_listens
    #print(to_replace)
    scaled_df_events = scaled_df_events.append(to_replace)
scaled_df_events.head()  """ 
#df_events.groupby('user').head()

"scaled_df_events = pd.DataFrame()\nfor user_id, group in df_events.groupby('user'):\n    #print(group)\n    min_listens = group['listens'].min()\n    max_listens = group['listens'].max()\n    std = (group['listens'] - min_listens) / (max_listens - min_listens)\n    scaled_listens = std * 999 + 1\n    to_replace = group.copy()\n    to_replace['listens'] = scaled_listens\n    #print(to_replace)\n    scaled_df_events = scaled_df_events.append(to_replace)\nscaled_df_events.head()  "

# Analysis of Alternating Least Squares with the Implicit Library

### Shaping the Data

In [64]:
#Normalize data to compare artist popularity vs recommendations
#Normalize artist popularity
normalized_artist_dist = pd.DataFrame(artist_dist)
normalized_artist_dist.columns = ['listens']
normalized_artist_dist['listens'] /= num_users
normalized_artist_dist.head()

num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0
#num_times_recommended.head()
#normalized_artist_dist.head()
#For potential use with graphs, not being used right now

In [65]:
def user_events_file_to_lil_matrix():
    # Artist to User matrix where artist_user_matrix[a, u] = num of times user u listened to artist a

    # 352805, 3000 (total artists, users)
    rows, cols = 352805, 3000
    artist_user_matrix = scipy.sparse.lil_matrix((rows, cols), dtype=int)

    # user	artist	album	track	timestamp

    user_dict = {}  # simplify user id to 1, 2, 3 ...
    artist_dict = {}

    # populate with user_events_file
    with open("/home/jimi/music_fairness/LFM-1b-surprise-and-implicit-analysis/ML_Base_Project/data/user_events.txt", 'r') as fp:
        line = fp.readline()
        loop_count = 0
        while line:
            # get data from line
            line = fp.readline()
            parts = line.split("\t")

            # end case
            try:
                user_id = int(parts[0])
                artist_id = int(parts[1])
            except ValueError:
                print("end of file " + line)
                break

            # use user_dict to shorten user_id
            if user_id not in user_dict:
                # this user_id has not bee seen
                user_dict[user_id] = len(user_dict)
            user_idx = user_dict[user_id]

            # use track_dict to shorten track_id
            if artist_id not in artist_dict:
                # this user_id has not bee seen
                artist_dict[artist_id] = len(artist_dict)
            artist_idx = artist_dict[artist_id]

            # increment count of user to track
            artist_user_matrix[artist_idx, user_idx] += 1

            # progress marker
            loop_count = loop_count + 1
            if loop_count % 10000000 == 0:
                print(str(loop_count) + "/ 28718087")  # / num of lines in file

    print(len(user_dict))
    print(len(artist_dict))

    # helpful dicts for converting artist and user count back to their ids
    user_count_to_id_dict = {v: k for k, v in user_dict.items()}
    artist_count_to_id_dict = {v: k for k, v in artist_dict.items()}

    return artist_user_matrix, user_dict, artist_dict, user_count_to_id_dict, artist_count_to_id_dict


## FUNCTIONS FROM from jmsteinw's blog post ## a_u_tuple


In [42]:
def make_train(ratings, pct_test=0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings,
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix.

    parameters:

    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix.

    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the
    training set for later comparison to the test set, which contains all of the original ratings.

    returns:

    training_set - The altered version of the original data with a certain percentage of the user-item pairs
    that originally had interaction set back to zero.

    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order
    compares with the actual interactions.

    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy()  # Make a copy of the original set to be the test set.
    test_set[test_set != 0] = 1  # Store the test set as a binary preference matrix
    training_set = ratings.copy()  # Make a copy of the original data we can alter as our training set.
    nonzero_inds = training_set.nonzero()  # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))  # Zip these pairs together of user,item index into list
    random.seed(0)  # Set the random seed to zero for reproducibility
    num_samples = int(
        np.ceil(pct_test * len(nonzero_pairs)))  # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples)  # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples]  # Get the user row indices
    item_inds = [index[1] for index in samples]  # Get the item column indices
    training_set[user_inds, item_inds] = 0  # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros()  # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds))  # Output the unique list of user rows that were altered


# Usage
# a_to_u_train, a_to_u_test, altered_users = make_train(csr_sparse_matrix, pct_test = 0.2)

In [43]:
a_u_tuple = user_events_file_to_lil_matrix()
a_u_matrix = a_u_tuple[0] #artist/user matrix
back_to_user_id = a_u_tuple[3] #dictionary to convert smaller user ids to original
back_to_artist_id = a_u_tuple[4]# same above but for artists
print("done")

10000000/ 28718087
20000000/ 28718087
end of file 
3000
352805
done


In [44]:
u_to_a_train, u_to_a_test, altered_users = make_train(a_u_matrix.T.tocsr(), pct_test=0.2) #makes training/test set
print("done")

done


### Training the Algorithm

In [45]:
# split original matrix into user vector and artist vector through ALS
user_vecs, artists_vecs = implicit.alternating_least_squares((u_to_a_train).astype('double'),iterations=50, factors=50)
print("done")

[15:35:08-140421]  This method is deprecated. Please use the AlternatingLeastSquares class instead
[15:35:08-140421]  GPU training requires factor size to be a multiple of 32. Increasing factors from 50 to 64.
[15:35:08-140421]  OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading


  0%|          | 0/50 [00:00<?, ?it/s]

done


In [46]:
print(np.array(user_vecs[0,:].dot(artists_vecs.T))[:10]) # gets first 5 recommendations

[0.7734145  0.9261155  0.50536424 0.83280516 0.9921204  0.84349346
 0.86699915 0.72439003 0.8522642  0.24458975]


### Get Top N Recommendations

In [47]:
def get_top_artists_implicit(user_vecs, artists_vecs, mf_train, back_to_user_id,back_to_artist_id, num=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        user_vecs: the feature vector for users
        mf_train: training matrix
        artists_vecs: the feature vector for artists
        back_to_user_id: dictionary to convert back to original user id
        back_to_artist_id: dictionary to convert back to original artist id
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id), ...] of size n.
    """
    top_artists_dict = defaultdict(list)
        # creating dictionary where user id is the key, and the val is a list of tuples of the artist id and 
        # the rating it thinks the user would give it
        
    for user_id in range(len(user_vecs)):
        
        pref_vec = mf_train[user_id,:].toarray() # Get the ratings from the training set ratings matrix
        pref_vec = pref_vec.reshape(-1)
        pref_vec =  pref_vec + 1 # Add 1 to everything, so that artists not listened to yet are equal to 1
        pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
        
        rec_vector = user_vecs[user_id,:].dot(artists_vecs.T) # Get dot product of user vector and all artist vectors
        # Scale this recommendation vector between 0 and 1
        
        min_max = MinMaxScaler()
        rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
        recommend_vector = pref_vec*rec_vector_scaled 
        # Items already purchased have their recommendation multiplied by zero
        
        product_idx = np.argsort(recommend_vector)[::-1][:num] # Sort the indices of the items into order 
        # of best recommendations
        rec_list = [] # start empty list to store items
        actual_user_id = back_to_user_id[user_id]
        #print(actual_user_id)
        
        for index in product_idx:
            actual_artist_id = back_to_artist_id[index]
            rec_list.append(actual_artist_id)
            
        top_artists_dict[actual_user_id] = rec_list
    print("done")
    return top_artists_dict
    
    

In [48]:
implixit_top_n = get_top_artists_implicit(user_vecs, artists_vecs, u_to_a_train,back_to_user_id, back_to_artist_id,num = 10) 
#gets top n artists for all the users in the dataset
print("done")

done
done


In [49]:
print(implicit_top_n[6532902]) #testing out getting top recommendations for a user

[2908, 15483, 6443, 43, 2836, 1998, 93, 3103, 6236, 2562]


In [50]:
print(artist_dist[6443]) #getting popularity of artist by indexing by id

180


### Calculating GAPr and Delta GAP for ALS

In [52]:
# Train and test the four algorithms on our data set
low_gap_r_list = []
medium_gap_r_list = []
high_gap_r_list = []
total_gap_r_list= []
#keeps track of the GAPr of each of the algs

alg_recommendations = [] #Keeps track of how many times each artist was recommended

#Will put for loop here with training, but for now we already have predictions
    
num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0 #resets for each algorithm
    
    
# resets to 0 before calculating for new alg
low_gap_r = 0
medium_gap_r = 0
high_gap_r = 0
    
#keeps track of group size since it is unpredictable since the test set is selected randomly
num_low_users = 0
num_medium_users = 0
num_high_users = 0
    
#keeps track of the mean absolute error of the current alg
low_mae = 0
medium_mae = 0
high_mae = 0
top_artists = implicit_top_n #Gets the top N recommendations for each user
    

for user_id, ratings in top_artists.items():
    artist_id_list = [] #user profile size to compute top fraction in GAPr
    for artist_id in ratings:
            
        artist_id_list.append(artist_id)
        num_times_recommended.loc[artist_id] += 1 #increments the number of times the artist was recommended

    gap = sum(artist_dist[artist_id_list]/ num_users) / len(artist_id_list) #fraction in numerator for GAPr
    #print("gap:" ,gap)
    if user_id in low_users.index: #summation of all of the top fractions to compute numerator for GAPr
        low_gap_r += gap
        num_low_users += 1 #increments group size to get the denominator to compute GAPr

    elif user_id in medium_users.index:
        medium_gap_r += gap
        num_medium_users += 1

    elif user_id in high_users.index:
        high_gap_r += gap
        num_high_users += 1
alg_recommendations.append(num_times_recommended)
            
total_gap_r = (low_gap_r + medium_gap_r + high_gap_r) / (num_low_users + num_medium_users + num_high_users)
low_gap_r = low_gap_r / num_low_users #Computing GAPr for each group size for each algorithm
medium_gap_r = medium_gap_r / num_medium_users
high_gap_r = high_gap_r / num_high_users

total_gap_r_list.append(total_gap_r)
low_gap_r_list.append(low_gap_r)
medium_gap_r_list.append(medium_gap_r)
high_gap_r_list.append(high_gap_r)

print("done")

done


In [53]:
#Computes change in GAP for each RecSys alg
delta_gap_low_list = []
delta_gap_medium_list = []
delta_gap_high_list = []
delta_gap_total_list = []

for i in range(len(low_gap_r_list)):
    delta_gap_low = ((low_gap_r_list[i] - low_gap_p) / low_gap_p)
    delta_gap_medium = ((medium_gap_r_list[i] - medium_gap_p) / medium_gap_p)
    delta_gap_high = ((high_gap_r_list[i] - high_gap_p) / high_gap_p)
    delta_gap_total = ((total_gap_r_list[i] - total_gap_p) / total_gap_p)
    
    delta_gap_low_list.append(delta_gap_low)
    delta_gap_medium_list.append(delta_gap_medium)
    delta_gap_high_list.append(delta_gap_high)
    delta_gap_total_list.append(delta_gap_total)
    
print(delta_gap_low_list[0])

print(delta_gap_medium_list[0])

print(delta_gap_high_list[0])

print(delta_gap_total_list[0])


1.5132151940425258
1.7332807395549104
1.7789633524148147
1.6850323399738416


### Calculating AUC for ALS

In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics.

    parameters:

    - predictions: your prediction output

    - test: the actual target result you are comparing to

    returns:

    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    # shuffle list of predictions (shuffle function)
    # shuffling dissassociates link to artist - > roc of .5 (random)
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered.

    parameters:

    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model

    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one.

    altered_users - The indices of the users where at least one user/item pair was altered from make_train function

    test_set - The test set constucted earlier from make_train function

    returns:

    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''

    store_auc = []  # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = []  # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis=0)).reshape(-1)  # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users:  # Iterate through each user that had an item altered
        training_row = training_set[user, :].toarray().reshape(-1)  # Get the training set row
        zero_inds = np.where(training_row == 0)  # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user, :]
        pred = user_vec.dot(item_vecs).toarray()[0, zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user, :].toarray()[0, zero_inds].reshape(-1)
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training
        pop = pop_items[zero_inds]  # Get the item popularity for our chosen items
        curr_auc_score = auc_score(pred, actual)
        store_auc.append(curr_auc_score)  # Calculate AUC for the given user and store
        curr_pop_score = auc_score(pop, actual)
        popularity_auc.append(curr_pop_score)  # Calculate AUC using most popular and score
        # print(user, "\t", curr_auc_score , "\t", curr_pop_score)
    # End users iteration

    return float('%.3f' % np.mean(store_auc)), float('%.3f' % np.mean(popularity_auc))
    # Return the mean AUC rounded to three decimal places for both test and popularity benchmark


In [ ]:
rec_auc, pop_auc = calc_mean_auc(u_to_a_train, altered_users,[sparse.csr_matrix(user_vecs), sparse.csr_matrix(artists_vecs.T)],
                                             u_to_a_test)
print(rec_auc)
print(pop_auc)

# Rectorch Training

### Setting Up The Data

In [14]:
data_events = df_events.copy(deep=True)
data_events.to_csv('data_events.csv',index=False,header=False )
print("done")

done


In [15]:
cfg_data_test = {
    "processing": {
        "data_path": "data_events.csv",
        "threshold": 0,
        "separator": ",",
        "header": None,
        "u_min": 50,
        "i_min": 50
    },
    "splitting": {
        "split_type": "horizontal",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 0.1,
        "test_size": 0.1,
        "test_prop": 0.2
    }
}



cfg_data_full = {
    "processing": {
        "data_path": "data_events.csv",
        "threshold": 0,
        "separator": ",",
        "header": None,
        "u_min": 0,
        "i_min": 0
    },
    "splitting": {
        "split_type": "horizontal",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 0.1,
        "test_size": 0.1,
        "test_prop": 0.2
    }
}

In [16]:
dataset = DataProcessing(cfg_data_test).process_and_split()
dataset

[18:30:55-280421]  Reading raw data file data_events.csv.
[18:31:00-280421]  Applying filtering.
[18:31:00-280421]  Filtered 988317 ratings.
[18:31:00-280421]  Shuffling data.
[18:31:00-280421]  Creating training, validation and test set.


Dataset(n_users=2804, n_items=5131, n_ratings=767044)

In [17]:

datasetFull = DataProcessing(cfg_data_full).process_and_split()
datasetFull

[18:31:01-280421]  Reading raw data file data_events.csv.
[18:31:06-280421]  Applying filtering.
[18:31:06-280421]  Shuffling data.
[18:31:06-280421]  Creating training, validation and test set.
[18:31:07-280421]  Skipped 25021 ratings in validation set.
[18:31:07-280421]  Skipped 24878 ratings in test set.


Dataset(n_users=3000, n_items=304953, n_ratings=1705462)

In [18]:
sparse_sampler = SparseDummySampler(dataset, mode="train")

### Grid Search on SLIM

In [25]:
lambda_vals =  np.array([0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3])
beta_vals =  np.array([0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3])


error = np.zeros([lambda_vals.size,beta_vals.size])

sparse_sampler = SparseDummySampler(dataset, mode="train")
for i in range(len(lambda_vals)):
    for j in range(len(beta_vals)):
        sparse_sampler.train()
        
        print(lambda_vals[i])
        print(beta_vals[j])
        
        slim = SLIM(l1_reg=lambda_vals[i], l2_reg=beta_vals[j])
        slim.train(sparse_sampler)
        
        sparse_sampler.test()
        
        results = evaluate(slim, sparse_sampler, ["ap@5000"])
        error[i,j] = collect_results(results).get("ap@5000")[0]
        print(collect_results(results))
        print(collect_results(results).get("ap@5000")[0])
        
print(error)
        
minError = np.argwhere(error == np.amax(error))
Lambda = lambda_vals[(minError[0][0])]
beta = beta_vals[(minError[0][1])]
print(Lambda)
print(beta)


0.5
0.5


[18:54:58-270421]  | item 513/5131 | ms/user 3.70 |
[18:55:00-270421]  | item 1026/5131 | ms/user 3.62 |
[18:55:02-270421]  | item 1539/5131 | ms/user 3.64 |
[18:55:03-270421]  | item 2052/5131 | ms/user 3.63 |
[18:55:05-270421]  | item 2565/5131 | ms/user 3.65 |
[18:55:07-270421]  | item 3078/5131 | ms/user 3.64 |
[18:55:09-270421]  | item 3591/5131 | ms/user 3.66 |
[18:55:11-270421]  | item 4104/5131 | ms/user 3.65 |
[18:55:13-270421]  | item 4617/5131 | ms/user 3.65 |
[18:55:15-270421]  | item 5130/5131 | ms/user 3.64 |
[18:55:15-270421]  | training complete | total training time 18.74 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@50000': (0.0, 0.0)}
0.0
0.5
1.0


[18:55:17-270421]  | item 513/5131 | ms/user 3.64 |
[18:55:18-270421]  | item 1026/5131 | ms/user 3.64 |
[18:55:20-270421]  | item 1539/5131 | ms/user 3.64 |
[18:55:22-270421]  | item 2052/5131 | ms/user 3.65 |
[18:55:24-270421]  | item 2565/5131 | ms/user 3.65 |
[18:55:26-270421]  | item 3078/5131 | ms/user 3.64 |
[18:55:28-270421]  | item 3591/5131 | ms/user 3.64 |
[18:55:30-270421]  | item 4104/5131 | ms/user 3.65 |
[18:55:32-270421]  | item 4617/5131 | ms/user 3.72 |
[18:55:33-270421]  | item 5130/5131 | ms/user 3.69 |
[18:55:33-270421]  | training complete | total training time 18.78 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@50000': (0.0, 0.0)}
0.0
0.5
1.5


...

{'ap@50000': (0.0, 0.0)}
0.0
1.0
3.5


[19:00:00-270421]  | item 513/5131 | ms/user 3.61 |
[19:00:01-270421]  | item 1026/5131 | ms/user 3.60 |
[19:00:03-270421]  | item 1539/5131 | ms/user 3.61 |
[19:00:05-270421]  | item 2052/5131 | ms/user 3.69 |
[19:00:07-270421]  | item 2565/5131 | ms/user 3.71 |
[19:00:09-270421]  | item 3078/5131 | ms/user 3.68 |
[19:00:11-270421]  | item 3591/5131 | ms/user 3.66 |
[19:00:13-270421]  | item 4104/5131 | ms/user 3.68 |
[19:00:15-270421]  | item 4617/5131 | ms/user 3.68 |
[19:00:17-270421]  | item 5130/5131 | ms/user 3.81 |
[19:00:17-270421]  | training complete | total training time 18.88 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@50000': (0.0, 0.0)}
0.0
1.0
4.0


[19:00:19-270421]  | item 513/5131 | ms/user 3.71 |
[19:00:20-270421]  | item 1026/5131 | ms/user 3.75 |
[19:00:22-270421]  | item 1539/5131 | ms/user 3.75 |


KeyboardInterrupt: 

### Training SLIM on Partial Dataset with Tuned Parameters

In [47]:
#Learned best Lambda = 0.0003 and best Beta = 0.03
sparse_sampler.train()
slim = SLIM(l1_reg=Lambda, l2_reg=beta)
slim.train(sparse_sampler)

[13:26:59-090421]  | item 513/5131 | ms/user 13.48 |
[13:27:06-090421]  | item 1026/5131 | ms/user 13.20 |
[13:27:13-090421]  | item 1539/5131 | ms/user 13.07 |
[13:27:19-090421]  | item 2052/5131 | ms/user 12.69 |
[13:27:25-090421]  | item 2565/5131 | ms/user 12.41 |
[13:27:32-090421]  | item 3078/5131 | ms/user 12.44 |
[13:27:38-090421]  | item 3591/5131 | ms/user 12.23 |
[13:27:44-090421]  | item 4104/5131 | ms/user 12.11 |
[13:27:50-090421]  | item 4617/5131 | ms/user 11.97 |
[13:27:56-090421]  | item 5130/5131 | ms/user 11.72 |
[13:27:56-090421]  | training complete | total training time 64.35 s |


In [48]:
sparse_sampler.test()
results = evaluate(slim, sparse_sampler, ["ap@5000"])
collect_results(results)

/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.1413211884342339, 0.08234902229175063)}

### Saving/Loading Model

In [49]:
slim.save_model("slimHorizontal")

[13:28:22-090421]  Saving SLIM model to slimHorizontal...
[13:28:22-090421]  Model saved!


In [32]:
slim2 = SLIM(l1_reg=.0003, l2_reg=.03)
slim2 = slim2.load_model("slimHorizontal")

[17:24:27-280421]  Loading SLIM model from slimHorizontal...
[17:24:27-280421]  Model loaded!


In [35]:
results = evaluate(slim2, sparse_sampler, ["ap@5000"])
collect_results(results)

[17:24:45-280421]  Sampler must be in valid or test mode. Froced switch to test mode!


{'ap@5000': (0.1413211884342339, 0.08234902229175063)}

### Getting Predictions Matrix

In [19]:
def getPredictions(model, sampler):
    """Returns the Predictions matrix for a rectorch model
    Args:
        model: The rectorch model to get predictions for
        sampler: the sampler the model used
        
    Returns:
        the predictions matrix for the model
    """
    sampler.test()
    for _, (data_input, ground_truth) in enumerate(sampler):
        data_input, ground_truth = prepare_for_prediction(data_input, ground_truth)
        predictions = model.predict(*data_input)[0].cpu().numpy()
    
    return predictions

In [41]:
predictions = getPredictions(slim2, sparse_sampler)

2804


### Getting Top N Artists for SLIM (Works with Any Rectorch Algorithm)

In [20]:
def get_top_artists(predictions, back_to_user_id,back_to_artist_id, num=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        Predictions: the Prediction vectors
        back_to_user_id: dictionary to convert back to original user id
        back_to_artist_id: dictionary to convert back to original artist id
        num(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id), ...] of size n.
    """
    top_artists_dict = defaultdict(list)
        # creating dictionary where user id is the key, and the val is a list of tuples of the artist id and 
        # the rating it thinks the user would give it
        
    for user_id in range(len(predictions)):
        
        rec_vector = predictions[user_id,:].copy()
        #print(rec_vector)
        
        recommend_vector = rec_vector
        
        product_idx = np.argsort(recommend_vector)[::-1][:num] # Sort the indices of the items into order 
        # of best recommendations
        rec_list = [] # start empty list to store items
        actual_user_id = back_to_user_id[user_id]
        #print(actual_user_id)
        
        for index in product_idx:
            actual_artist_id = back_to_artist_id[index]
            rec_list.append(actual_artist_id)
            
        top_artists_dict[actual_user_id] = rec_list
    print("done")
    return top_artists_dict
    
    

#### Reversing the matrices given by the dataset so we can get the original artist and user ids

In [21]:
def get_reverse_matrices(dataset):
    """Simply reverses the dictionaries in a rectorch dataset to be used to 
    reference back to the original ids"""
    id2u =  dict(map(reversed, dataset.u2id.items())) #dict for putting user ids back to normal
    id2i = dict(map(reversed, dataset.i2id.items())) #dict for translating artist ids back to normal
    return id2u, id2i

In [58]:
id2u, id2i = get_reverse_matrices(dataset)

In [59]:
rectorch_top_n = get_top_artists(predictions, id2u, id2i)

done


In [47]:
rectorch_top_n[6532902]

[2836, 17292, 6443, 17302, 35098, 7941, 7463, 1804, 15629, 18830]

### Calculating GAPr and Delta GAP for Rectorch Algorithms (SLIM)

In [22]:
def calcGapR(algs_to_test, samplers, dataset, artist_dist, get_top_artists, get_reverse_matrices, get_predictions):
    """Returns the list of GAPrs for all categories given a list of algorithms
        ALGORITHMS MUST ALREADY BE TRAINED SEPARATELY TO CONSERVE TIME
        Args:
            algs_to_test: list of already trained models
            samplers: the sampler used for each algorithm (must be same length as algs_to_test)
            dataset: the rectorch dataset object that everything is trained on
            artist_dist: pandas df of how many times each artist was listened to by unique users
            get_top_artists: function to get top n artists for each alg
            get_reverse_matrices: function to reverse matrices to get user and artist ids back to their original form
            get_predictions: function to get predictions for the models of each alg
            
        Returns:
            low_gap_r_list: list of GAPr for low pop group
            medium_gap_r_list: list of GAPr for med pop group
            high_gap_r_list: list of GAPr for high pop group
            total_gap_r_list: list of GAPr across entire dataset
            all indexed in the same order as algs_to_test"""
    # Train and test the four algorithms on our data set
    low_gap_r_list = []
    medium_gap_r_list = []
    high_gap_r_list = []
    total_gap_r_list= []
    #keeps track of the GAPr of each of the algs

    alg_recommendations = [] #Keeps track of how many times each artist was recommended

    #Will put for loop here with training, but for now we already have predictions
    for i in range(len(algs_to_test)):   

        num_times_recommended = pd.DataFrame(artist_dist)
        num_times_recommended.columns = ['Recommendation Frequency']
        num_times_recommended['Recommendation Frequency'] = 0 #resets for each algorithm


        # resets to 0 before calculating for new alg
        low_gap_r = 0
        medium_gap_r = 0
        high_gap_r = 0

        #keeps track of group size since it is unpredictable since the test set is selected randomly
        num_low_users = 0
        num_medium_users = 0
        num_high_users = 0

        #keeps track of the mean absolute error of the current alg
        low_mae = 0
        medium_mae = 0
        high_mae = 0

        predictions = get_predictions(algs_to_test[i], samplers[i])
        userBack, artistBack = get_reverse_matrices(dataset)
        top_artists = get_top_artists(predictions, userBack, artistBack) #Gets the top N recommendations for each user


        for user_id, ratings in top_artists.items():
            artist_id_list = [] #user profile size to compute top fraction in GAPr
            for artist_id in ratings:

                artist_id_list.append(artist_id)
                num_times_recommended.loc[artist_id] += 1 #increments the number of times the artist was recommended

            gap = sum(artist_dist[artist_id_list]/ num_users) / len(artist_id_list) #fraction in numerator for GAPr
            #print("gap:" ,gap)
            if user_id in low_users.index: #summation of all of the top fractions to compute numerator for GAPr
                low_gap_r += gap
                num_low_users += 1 #increments group size to get the denominator to compute GAPr

            elif user_id in medium_users.index:
                medium_gap_r += gap
                num_medium_users += 1

            elif user_id in high_users.index:
                high_gap_r += gap
                num_high_users += 1
        alg_recommendations.append(num_times_recommended)

        total_gap_r = (low_gap_r + medium_gap_r + high_gap_r) / (num_low_users + num_medium_users + num_high_users)
        low_gap_r = low_gap_r / num_low_users #Computing GAPr for each group size for each algorithm
        medium_gap_r = medium_gap_r / num_medium_users
        high_gap_r = high_gap_r / num_high_users

        total_gap_r_list.append(total_gap_r)
        low_gap_r_list.append(low_gap_r)
        medium_gap_r_list.append(medium_gap_r)
        high_gap_r_list.append(high_gap_r)

    return low_gap_r_list, medium_gap_r_list, high_gap_r_list, total_gap_r_list

    print("done")

In [23]:
#Computes change in GAP for each RecSys alg
def calc_delta_gap(gap_p_list, gap_r_list):
    """Computes delta gap of all recsys data recorded given the gap_r_list, should be in order of algs_list if used
        with function above

        Args:
            gap_p_list: list of the computed gapPs (in order of low,med,high,total)
            gap_r_list: list of the computed gapRs (in order of low,med,high,total)
        Returns:
            delta_gap_low_list: list of all computed delta gaps for all algs for the low user group
            delta_gap_medium_list: list of all computed delta gaps for all algs for the med user group
            delta_gap_high_list: list of all computed delta gaps for all algs for the high user group
            delta_gap_total_list: list of all computed delta gaps for all algs for all users
    """
    delta_gap_low_list = []
    delta_gap_medium_list = []
    delta_gap_high_list = []
    delta_gap_total_list = []

    low_gap_p = gap_p_list[0]
    medium_gap_p = gap_p_list[1]
    high_gap_p = gap_p_list[2]
    total_gap_p = gap_p_list[3]

    low_gap_r_list = gap_r_list[0]
    medium_gap_r_list = gap_r_list[1]
    high_gap_r_list = gap_r_list[2]
    total_gap_r_list = gap_r_list[3]

    for i in range(len(low_gap_r_list)):
        delta_gap_low = ((low_gap_r_list[i] - low_gap_p) / low_gap_p)
        delta_gap_medium = ((medium_gap_r_list[i] - medium_gap_p) / medium_gap_p)
        delta_gap_high = ((high_gap_r_list[i] - high_gap_p) / high_gap_p)
        delta_gap_total = ((total_gap_r_list[i] - total_gap_p) / total_gap_p)

        delta_gap_low_list.append(delta_gap_low)
        delta_gap_medium_list.append(delta_gap_medium)
        delta_gap_high_list.append(delta_gap_high)
        delta_gap_total_list.append(delta_gap_total)

    
    return delta_gap_low_list, delta_gap_medium_list, delta_gap_high_list, delta_gap_total_list

In [66]:
low_gap_r_list, medium_gap_r_list,high_gap_r_list,total_gap_r_list = calcGapR([slim2],[sparse_sampler], dataset, artist_dist, get_top_artists, get_reverse_matrices, getPredictions)

calc_delta_gap([low_gap_p, medium_gap_p,high_gap_p, total_gap_p],[low_gap_r_list,medium_gap_r_list,high_gap_r_list,total_gap_r_list])

done
2.663266993343709
2.720945436349978
2.409118205863711
2.5877539912773777


([2.663266993343709],
 [2.720945436349978],
 [2.409118205863711],
 [2.5877539912773777])

In [25]:
sparse_sampler_full = SparseDummySampler(datasetFull, mode="train")

In [20]:
sparse_sampler_full.train()
slimFull = SLIM(l1_reg=.0003, l2_reg=.03)
slimFull.train(sparse_sampler_full)

sparse_sampler_full.test()
results = evaluate(slimFull, sparse_sampler_full, ["ap@5000"])
collect_results(results)

slimFull.save_model("slim_full")

[19:24:04-140421]  | item 30495/304953 | ms/user 338.65 |
[22:14:27-140421]  | item 60990/304953 | ms/user 335.23 |
[01:06:36-150421]  | item 91485/304953 | ms/user 338.72 |
[03:59:30-150421]  | item 121980/304953 | ms/user 340.17 |
[06:52:59-150421]  | item 152475/304953 | ms/user 341.34 |
[09:48:10-150421]  | item 182970/304953 | ms/user 344.68 |
[12:44:32-150421]  | item 213465/304953 | ms/user 347.00 |
[15:44:01-150421]  | item 243960/304953 | ms/user 353.15 |
[18:44:57-150421]  | item 274455/304953 | ms/user 355.98 |
[21:47:56-150421]  | item 304950/304953 | ms/user 360.02 |
[21:48:00-150421]  | training complete | total training time 105362.39 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth
[21:50:09-150421]  Saving SLIM model to slim_full...
[21:50:15-150421]  Model saved!


In [26]:
slimFull = SLIM(l1_reg=.0003, l2_reg=.03)
slimFull = slimFull.load_model("slim_full")

[18:31:37-280421]  Loading SLIM model from slim_full...
[18:31:42-280421]  Model loaded!


In [27]:
low_gap_r_list, medium_gap_r_list,high_gap_r_list,total_gap_r_list = calcGapR([slimFull],[sparse_sampler_full], datasetFull, artist_dist, get_top_artists, get_reverse_matrices, getPredictions)


done
2.723715030009914
2.5596250781661016
2.141520294075726
2.450969770363109


([2.723715030009914],
 [2.5596250781661016],
 [2.141520294075726],
 [2.450969770363109])

In [28]:
calc_delta_gap([low_gap_p, medium_gap_p,high_gap_p, total_gap_p],[low_gap_r_list,medium_gap_r_list,high_gap_r_list,total_gap_r_list])

done


In [28]:
array_sampler_full = ArrayDummySampler(datasetFull, mode="train")
array_sampler_test = ArrayDummySampler(dataset,mode="train")

In [31]:
ease = EASE(500)
ease.train(array_sampler_test)

array_sampler_test.test()
results = evaluate(ease, array_sampler_test, ["ap@5000"])
collect_results(results)


[19:56:14-270421]  EASE - start tarining (lam=500.0000)
[19:56:14-270421]  EASE - linear kernel computed
[19:56:17-270421]  EASE - training complete


{'ap@5000': (0.13180846894672282, 0.08078918435269586)}

In [30]:
lambda_vals =  np.array([100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000])

error = np.zeros([lambda_vals.size])


for i in range(len(lambda_vals)):
        array_sampler_test.train()
        
        print(lambda_vals[i])
        
        ease = EASE(lambda_vals[i])
        ease.train(array_sampler_test)
        
        array_sampler_test.test()
        
        results = evaluate(ease, array_sampler_test, ["ap@5000"])
        error[i] = collect_results(results).get("ap@5000")[0]
        print(collect_results(results))
        print(collect_results(results).get("ap@5000")[0])
        
print(error)
        
minError = np.argwhere(error == np.amax(error))
Lambda = lambda_vals[(minError[0])]
print(Lambda)


[19:54:40-270421]  EASE - start tarining (lam=100.0000)


100


[19:54:40-270421]  EASE - linear kernel computed
[19:54:42-270421]  EASE - training complete
[19:54:44-270421]  EASE - start tarining (lam=150.0000)


{'ap@5000': (0.1179903148670593, 0.0778611649631584)}
0.1179903148670593
150


[19:54:44-270421]  EASE - linear kernel computed
[19:54:46-270421]  EASE - training complete
[19:54:48-270421]  EASE - start tarining (lam=200.0000)


...

{'ap@5000': (0.13017729393471678, 0.07946441590788961)}
0.13017729393471678
950


[19:55:47-270421]  EASE - linear kernel computed
[19:55:49-270421]  EASE - training complete
[19:55:50-270421]  EASE - start tarining (lam=1000.0000)


{'ap@5000': (0.12984975869899804, 0.07938022028209238)}
0.12984975869899804
1000


[19:55:51-270421]  EASE - linear kernel computed
[19:55:53-270421]  EASE - training complete


{'ap@5000': (0.1294967565823914, 0.07918547487211418)}
0.1294967565823914
[0.11799031 0.12337435 0.12649232 0.12857188 0.12988903 0.13084361
 0.13139022 0.13160642 0.13180847 0.13168838 0.1316456  0.13141524
 0.13113943 0.13102867 0.13073165 0.13045312 0.13017729 0.12984976
 0.12949676]
[500]


In [21]:
!jupyter nbconvert --to script Research.ipynb

[NbConvertApp] Converting notebook Research.ipynb to script
[NbConvertApp] Writing 36219 bytes to Research.py


In [ ]:
# 500 is best lambda

In [31]:
rnd = Random(datasetFull.n_items)
rnd.train(array_sampler_full)
pop = Popularity(datasetFull.n_items)
pop.train(array_sampler_full)

In [34]:
low_gap_r_list, medium_gap_r_list,high_gap_r_list,total_gap_r_list = calcGapR([rnd,pop],[array_sampler_full,array_sampler_full], datasetFull, artist_dist, get_top_artists, get_reverse_matrices, getPredictions)
d_gap_low, d_gap_med, d_gap_high, d_gap_tot = calc_delta_gap([low_gap_p, medium_gap_p,high_gap_p, total_gap_p],[low_gap_r_list,medium_gap_r_list,high_gap_r_list,total_gap_r_list])


done
done
-0.9631437682399542
-0.9670995676674233
-0.9702689870631357
-0.9671168964772376


In [ ]:
print("Rand \t Pop")
print(d_gap_low[0] + "\t" + d_gap_low[1])
print("---")
print(d_gap_med[0] + "\t" + d_gap_med[1])
print("---")
print(d_gap_high[0] + "\t" + d_gap_high[1])
print("---")
print(d_gap_tot[0] + "\t" + d_gap_tot[1])
print("---")